In [3]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Math
import sympy as sym
import scipy as sci
from scipy import linalg
from IPython.display import display, Latex
from sympy import matrices


x, y, z,p,q,r,s,t, a,b,c,d,e,f,g,h,i = sym.symbols('x y z p q r s t a b c d e f g h i')
a0,a1,a2,a3,a4,a5,a6,a7,a8,x9 = sym.symbols('a:10')
b0,b1,b2,b3,b4,b5,b6,b7,b8,b9 = sym.symbols('b:10')
x0,x1,x2,x3,x4,x5,x6,x7,x8,x9 = sym.symbols('x:10')
y0,y1,y2,y3,y4,y5,y6,y7,y8,y9 = sym.symbols('y:10')
z0,z1,z2,z3,z4,z5,z6,z7,z8,z9 = sym.symbols('z:10')
λ = sym.symbols('λ')
pm = np.array([+1, -1])
arr = lambda A: sym.Matrix(np.array(A))
invV= lambda v: arr(v[::-1])

#elementary_row_op(op='n->kn', row=None, k=None, row1=None, row2=None)
#“n->kn” (row n goes to k*n)        “n<->m” (swap row n and row m)          “n->n+km” (row n goes to row n + k*row m)
opR = sym.matrices.matrices.MatrixReductions.elementary_row_op

sym.init_printing()

In [6]:
#n= lambda : [i for i in enumerate([int(i) if i.isnumeric() else i for i in input()])]
n= lambda s = '' : np.array([int(i) if i.isnumeric() ==True else sym.sympify(i)  for i in input(s)]   )       
#neg = lambda : np.array([int(i) * int(input('mulitply by negatives')) for i in n('insert values')])



# Dot product v*w = v1*w1 + v2*w2 +...+ vn*wn
dotv = lambda v, w: sum(v*w)

#Length of vector ||v|| = √v1^2 +v2^2+...+vn^2
lenv = lambda v: np.sqrt(sum(v**2))

#unit vector in same dir as u = v / ||v||
unitv = lambda v:  v/lenv(v)

#cosθ = u*U where both are unit vectors or v*w / ||v||*||w||
cosθ = lambda v, w: dotv(unitv(v), unitv(w))

#Converts resulting radians into degrees
degθ = lambda θ : np.arccos(θ)* 180 / np.pi

#schwarz inequality |v*w| <= ||v||*||w||
#truthSch = 
schwarz =  (lambda v,w : f'{round(dotv(v,w),3)} <= {round(lenv(v) * lenv(w), 3)}'
                        if dotv(v,w) <= lenv(v) * lenv(w) 
                        else False)

#triangle inequality ||v + w|| <= ||v||+||w||

triIneq =  (lambda v,w :f'{round(lenv(v+w),3)} <= {round(lenv(v) + lenv(w), 3)}'
                        if lenv(v+w) <= lenv(v) + lenv(w) 
                        else False)


#concatenates vectors to an array
Avw = lambda v,w : np.array([v,w])

#  det[[a,b],[c,d]]  = ad-bc
detv  = lambda v,w: ((v * w[::-1])[0]) - ((v * w[::-1])[1])

detA2  = lambda A : (A[0,:][0] * A[1,::-1][0]) -  (A[0,:][1] * A[1,::-1][1])

detA3 = lambda A : sum([A[0]*detA2(A[[1,2],[1,2]]) , A[1]*detA2(A[[1,2],[0,2]]),  A[2]*detA2(A[[1,2],[0,1]])] * cofactors(np.array(A))[0,:])

#detA4plus = lambda A : 


#I = lambda A: [[j for j in  i ]for i in  range(len(A))]
#I = lambda A: np.array([[ 1 if i[0] == j[0] else 0 for j in enumerate(A[i[0]])] for i in enumerate(A)])
I = lambda n: sym.Matrix(n,n,( lambda i,j: 1 if i == j else 0))
#Cross product for 2 vectors w/ len 3
# u= [a, b, c], 
# v= [d, e, f] ; u x v = [bf - ce, af - cd, ae - bd]
# or [ detA( without col 1), - detA( w/o col 2), detA( w/o col 3) ]  
cross = lambda v,w:    np.array(
                        [detA2(np.stack(( Avw(v,w)[::,1], Avw(v,w)[::,2]),axis = 1)),
                        detA2(np.stack(( Avw(v,w)[::,0], Avw(v,w)[::,2]),axis = 1)),
                        detA2(np.stack(( Avw(v,w)[::,0], Avw(v,w)[::,1]),axis = 1))
                        ])#* cofactors(Avw(v,w))

crossv= lambda v,w : cofactors(Avw(v,w))[0] * cross(v,w) 


#cofactor given by taking the index of A_ij and evaluating as C_ij = (-1)^i+j resulting in meagtive or posivte one

cofactors = lambda A:  (-1)**np.array([[sum(np.array([i[0]+1,j[0]+1]))
                        for j in enumerate(A[i[0]])] 
                        for i in enumerate(A)])
                        
                        
#trace = sum of the diagonals
tr = lambda A : sum([(A[i,i]) for i in range(len(A[0,::])) ])



#Char Polynomial p(λ)=λ2−tr(A)λ+det(A), returns sympy polynomial object 
cpR2 = lambda A: sym.Poly((λ**2 - tr(A)*λ + detA2(A)),λ) 
#CP= −λ^3 + tr(A) λ^2 + 1/2 (tr(A)^2 − tr(A^2))λ + det(A)
cpR3 = lambda A: sym.Poly((  -λ**3 + tr(A) * λ**2 + 1/2 * (tr(A)**2 - tr(A**2) ) *λ + detA3(A)))

# -b form to find roots MUST come as list [a,b,c] use negB(P.coeffs())
negB = lambda P : (-P[1] + pm *  sym.sqrt(P[1]**2 - 4*P[0]*P[2]) )/ 2*P[0]

AλI = lambda A: (A-λ*I(A))

EigenValues = lambda A : negB(cpR2(A).coeffs()) 

EigenMatrix= lambda A : AλI(A)*arr([a,b])#.subs(λ, [i for i in EigenValues(A)])

EigenPolys = lambda A: [[(EigenMatrix(A)[0][0] + EigenMatrix(A)[0][1]).subs(λ, i) ,
                        (EigenMatrix(A)[1][0] + EigenMatrix(A)[1][1]).subs(λ,i)] 
                        for i in EigenValues(A)]




In [7]:
cpR3(A), detA3(A), tr(A)**2, tr(A**2)
I(3)

NameError: name 'A' is not defined

In [64]:
#defines matrix with values, sign and shape
Amat  = lambda : (np.reshape(np.asarray(n('input array integers'))  * 
                                    [ i if i == 1 
                                        else -1 
                                        for i in n('input 1 for pos, 0 for neg')],
                            n('input array shape'))) 


#Takes matrices A,B  gets the dot product from row A * col B
dotA = lambda A,B : np.array([[dotv(A[i] , B[::,j]) for j in range(B.shape[1])] for i in range(A.shape[0])])

#dotA = lambda A,B : np.array(  [[dotv(A[0],B[::,0]), dotv(A[0],B[::,1]), dotv(A[0],B[::,2])],
                            #    [dotv(A[1],B[::,0]), dotv(A[1],B[::,1]), dotv(A[1],B[::,2])]])

#takes a 3x3 matrix [[a,b,c],
#                    [d,e,f],multi-augmented row reduction.(dh - eg)
detA3 = lambda A:  np.sum((cross(A[1],A[2]) * A[0]) * cofactors(A))

#Transpose: takes matrix of shape m x n and rewrites with columns in rows with shape n x m
transA = lambda A : np.array([[A[::,i][j] for j in range(A.shape[0])] for i in range(A.shape[1])])

#matrix of minors for A = m x n is found by filling each postion withthe det excluding its own col and row
# i.e   [[a,b,c],
#        [d,e,f],
#        [g,h,i]] pos a = det([ef][hi]), b= det([dg][fi]),...,i=det([ab][de]) 
minorA = lambda A : np.array([cross(A[1],A[2]), cross(A[0],A[2]), cross(A[0],A[1])] ) 

#adjoint is the matrix of minors of the transpose of A times the cofactors 
adjA = lambda A: minorA(transA(A)) * cofactors(A)

#The inverse of A; A^-1 = adjA/detA 
#given here as a list showing det , adjA
invA = lambda A : [ detA3(A), adjA(A) ]

#and here as the calculated matrix
invAcalc = lambda A :  adjA(A)/detA3(A)


In [65]:
M = Amat()

In [89]:
M[3,1],M[3,3]=-12,12
M = arr(M)
ref(M)
M.echelon_form()

1 0


⎛⎡1  -3   3   0 ⎤  ⎡1   0  0  0⎤⎞
⎜⎢              ⎥  ⎢           ⎥⎟
⎜⎢0   0   -5  5 ⎥  ⎢-3  1  0  0⎥⎟
⎜⎢              ⎥, ⎢           ⎥⎟
⎜⎢2  -6   3   3 ⎥  ⎢0   0  1  0⎥⎟
⎜⎢              ⎥  ⎢           ⎥⎟
⎝⎣4  -12  0   12⎦  ⎣0   0  0  1⎦⎠

2 0


⎛⎡1  -3   3   0 ⎤  ⎡1   0  0  0⎤⎞
⎜⎢              ⎥  ⎢           ⎥⎟
⎜⎢0   0   -5  5 ⎥  ⎢0   1  0  0⎥⎟
⎜⎢              ⎥, ⎢           ⎥⎟
⎜⎢0   0   -3  3 ⎥  ⎢-2  0  1  0⎥⎟
⎜⎢              ⎥  ⎢           ⎥⎟
⎝⎣4  -12  0   12⎦  ⎣0   0  0  1⎦⎠

3 0


⎛⎡1  -3   3   0 ⎤  ⎡1   0  0  0⎤⎞
⎜⎢              ⎥  ⎢           ⎥⎟
⎜⎢0  0   -5   5 ⎥  ⎢0   1  0  0⎥⎟
⎜⎢              ⎥, ⎢           ⎥⎟
⎜⎢0  0   -3   3 ⎥  ⎢0   0  1  0⎥⎟
⎜⎢              ⎥  ⎢           ⎥⎟
⎝⎣0  0   -12  12⎦  ⎣-4  0  0  1⎦⎠

2 1
3 2


⎛⎡1  -3  3   0⎤  ⎡1  0  0   0⎤⎞
⎜⎢            ⎥  ⎢           ⎥⎟
⎜⎢0  0   -5  5⎥  ⎢0  1  0   0⎥⎟
⎜⎢            ⎥, ⎢           ⎥⎟
⎜⎢0  0   -3  3⎥  ⎢0  0  1   0⎥⎟
⎜⎢            ⎥  ⎢           ⎥⎟
⎝⎣0  0   0   0⎦  ⎣0  0  -4  1⎦⎠

⎡1  -3  3   0⎤
⎢            ⎥
⎢0  0   -5  5⎥
⎢            ⎥
⎢0  0   -3  3⎥
⎢            ⎥
⎣0  0   0   0⎦

⎡1  -3  3   0⎤
⎢            ⎥
⎢0  0   -5  5⎥
⎢            ⎥
⎢0  0   0   0⎥
⎢            ⎥
⎣0  0   0   0⎦

In [145]:
def elBuild(n, i=0, j=0, x=1): 
    el = I(n)
    el[i,j] = x
    #display(el)
    return el

def elSwap(n, i,j) :
    el = elBuild(n)
    el.elementary_row_op(op='n<->m', row1=i,row=j)
    return el
    

def ref(A):
    lenA = len(A[:,0] )-1
    #piv = A[0,0], A[0,:]
    n = len(A[0,:])
    m = len(A[:,0])
    minDim = min(n,m)
    display(A)
    # Let's do forward step first.
    # at the end of this for loop, the matrix is in Row-Echelon form.
   

    for c in range(minDim):
        
        # every iteration, ignore one more row and column
        for r in range(c, minDim):
            piv=A[c,c], A[r,:]
            # find the first row with a nonzero entry in first column
            zero_row = A[r,c] == 0
            print (zero_row, r,c, A[r,c])
            if zero_row:
                print('ctd')
                #continue
                return A

            for i in range(c,minDim):
                E = elBuild(minDim,r,c,- A[r,c]/piv[0])
                A = E*A
                display((A,E))
            
            else:#  A[r,c] == 0:
                #A[[j,:]] = A[[j+1,:]]
                #print(i,j)
                E = elSwap(lenA+1,r,c)
                A = E*A#A.elementary_row_op(op='n<->m', row1=i,row=j)
                display((A,E))
            print(piv[0])

    '''for j in range(lenA): 
        
        piv=A[j,j], A[j,:]
        #print (piv)
        
        i = 0
        
        #print(f'j: {j}')
        while i < lenA - j :
            i += 1+j
            
            print(i,j)
            #print(A[i,j])
            if piv[0] != 0 :
                

                E = elBuild(lenA+1,i,j,- A[i,j]/piv[0])
                A = E*A
                display((A,E))
                
            elif  A[i,j] != 0:
                #A[[j,:]] = A[[j+1,:]]
                #print(i,j)
                E = elSwap(lenA+1,i,j)
                A = E*A#A.elementary_row_op(op='n<->m', row1=i,row=j)
                display((A,E))
                #print(A[i,j], A)
                #break'''
                
    return display(A)

ref(A)

⎡1  2  3⎤
⎢       ⎥
⎢3  6  2⎥
⎢       ⎥
⎣2  2  1⎦

False 0 0 1


⎛⎡-1  -2  -3⎤  ⎡-1  0  0⎤⎞
⎜⎢          ⎥  ⎢        ⎥⎟
⎜⎢3   6   2 ⎥, ⎢0   1  0⎥⎟
⎜⎢          ⎥  ⎢        ⎥⎟
⎝⎣2   2   1 ⎦  ⎣0   0  1⎦⎠

1
False 1 0 3


⎛⎡-1  -2  -3⎤  ⎡1  0  0⎤⎞
⎜⎢          ⎥  ⎢       ⎥⎟
⎜⎢0   0   -7⎥, ⎢3  1  0⎥⎟
⎜⎢          ⎥  ⎢       ⎥⎟
⎝⎣2   2   1 ⎦  ⎣0  0  1⎦⎠

-1
False 2 0 2


⎛⎡-1  -2  -3⎤  ⎡1  0  0⎤⎞
⎜⎢          ⎥  ⎢       ⎥⎟
⎜⎢0   0   -7⎥, ⎢0  1  0⎥⎟
⎜⎢          ⎥  ⎢       ⎥⎟
⎝⎣0   -2  -5⎦  ⎣2  0  1⎦⎠

-1
True 1 1 0
ctd


⎡-1  -2  -3⎤
⎢          ⎥
⎢0   0   -7⎥
⎢          ⎥
⎣0   -2  -5⎦

In [29]:
A= arr([[1,2,3],[3,6,2],[2,2,1]])
M = sym.Matrix([[1, 0, 0], [-3, 1, 0], [0, 0, 1]])
A


#A[:,0]
A[1,0], [i for i in range(len(A[:,0]))]
I(3), 
a = elBuild(3,1,0,-2)
A[0,:]
A,M, M*A #sym.DotProduct(A,M)

Matrix([[1, 0, 0], [-2, 1, 0], [0, 0, 1]])


⎛⎡1  2  3⎤  ⎡1   0  0⎤  ⎡1  2  3 ⎤⎞
⎜⎢       ⎥  ⎢        ⎥  ⎢        ⎥⎟
⎜⎢3  6  2⎥, ⎢-3  1  0⎥, ⎢0  0  -7⎥⎟
⎜⎢       ⎥  ⎢        ⎥  ⎢        ⎥⎟
⎝⎣2  2  1⎦  ⎣0   0  1⎦  ⎣2  2  1 ⎦⎠

In [127]:
def like_a_gauss(mat):
	"""
	Changes mat into Reduced Row-Echelon Form.
	"""
	n = len(mat[0,:])
	m = len(mat[:,0])
	minDim = min(n,m)
	display(mat)
	# Let's do forward step first.
	# at the end of this for loop, the matrix is in Row-Echelon format.
	for c in range(minDim):
		# every iteration, ignore one more row and column
		for r in range(c, minDim):
			# find the first row with a nonzero entry in first column
			zero_row = mat[r,c] == 0
			print (zero_row, r,c, mat[r][c])
			if zero_row:
				print('ctd')
				continue
			# swap current row with first row
			mat[c], mat[r] = mat[r], mat[c]
			# add multiples of the new first row to lower rows so lower
			# entries of first column is zero
			first_row_first_col = mat[c][c]
			for rr in range(c + 1, len(mat)):
				this_row_first = mat[rr][c]
				scalarMultiple = -1 * this_row_first / first_row_first_col
				for cc in range(c, len(mat[0])):
					mat[rr][cc] += mat[c][cc] * scalarMultiple
			break

	# At the end of the forward step
	print(mat)
	# Now reduce
	for i in range(min(len(mat), len(mat[0])) - 1, -1, -1):
		# divide last non-zero row by first non-zero entry
		first_elem_col = -1
		first_elem = -1
		for c in range(len(mat[0])):
			if mat[i][c] == 0:
				continue
			if first_elem_col == -1:
				first_elem_col = c
				first_elem = mat[i][c]
			mat[i][c] /= first_elem
		# add multiples of this row so all numbers above the leading 1 is zero
		for r in range(i):
			this_row_above = mat[r][first_elem_col]
			scalarMultiple = -1 * this_row_above
			for cc in range(len(mat[0])):
				mat[r][cc] += mat[i][cc] * scalarMultiple
		# disregard this row and continue
	print(mat)


def augment_that_sucker(mat1, mat2):
	"""
	Duct-tape mat2's columns to the right of mat1
	Return a new matrix.
	"""
	retval = []
	for i in range(len(mat1)):
		r = mat1[i]
		newrow = r[:] + mat2[i]
		retval.append(newrow)
	return np.array(retval)

def from_vector(vector):
	"""
	Convert a vector into a column matrix.
	"""
	retval = []
	for r in vector:
		retval.append([r])
	return np.array(retval)

def transpose(mat):
	"""
	Return a transposed version of mat.
	"""
	retval = []
	for c in range(len(mat[0])):
		newrow = []
		for r in range(len(mat)):
			newrow.append(mat[r][c])
		retval.append(newrow)
	return retval

# testcase from http://reference.wolfram.com/mathematica/ref/RowReduce.html

mattest = np.array([[1,2,3],[5,6,7],[7,8,9]])

mattest2 = from_vector([1,1,1])

mattest3 = augment_that_sucker(mattest, mattest2)			

like_a_gauss(mattest3)

print(transpose(mattest3)[2])
#like_a_gauss(A)



array([[ 2,  3,  4],
       [ 6,  7,  8],
       [ 8,  9, 10]])

False 0 0 2
False 1 1 -2
True 2 2 0
ctd
[[ 2  3  4]
 [ 0 -2 -4]
 [ 0  0  0]]
[[ 1  0 -1]
 [ 0  1  2]
 [ 0  0  0]]
[-1, 2, 0]


In [116]:
#len(), len(A[:0]),np.array(mattest)[0,:]
A[0,:],A[:,0]

⎛           ⎡1⎤⎞
⎜           ⎢ ⎥⎟
⎜[1  2  3], ⎢3⎥⎟
⎜           ⎢ ⎥⎟
⎝           ⎣2⎦⎠

In [ ]:
#A = np.array([[1,2],[3,2]])


#EigenMatrix(A)[0],sym.Poly(AλI(A)[0],a),sym.Poly(EigenMatrix(A)[0],a), EigenValues(A)
#EigenValues(A),EigenPolys(A),charPolyR2(A).coeffs()

#sym.linsolve((EigenPolys(A)[0], (0,0) ),(a,b))#, #EigenPolys(A)[0]

#[ [sym.solveset(j) for j in i]  for i in EigenPolys(A)]

A #sym.linear_eq_to_matrix(EigenPolys(A)[0],[a,b])
#sym.linsolve((EigenMatrix(A)),[a,b])#,EigenPolys(A)[0]
#print(f'{EigenValues(A)}\n,')
#print(f'{cpR2(A)}\n, {AλI(A)}')
#EigenMatrix(A)
#EigenPolys(A)#AλI(A)#cpR2(A)
#detA2(A),tr(A),
#A[0],
detA3(A),type(A)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
AλI(A)#AλI(A)*arr([a,b])

array([[1 - λ, 2],
       [3, 2 - λ]], dtype=object)

In [ ]:
sym.solvers.linsolve

<function sympy.solvers.solveset.linsolve(system, *symbols)>

In [ ]:
A = arr([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
b = arr([3, 6, 9])
sym.linsolve((sym.Matrix(A),sym.Matrix(b)), [x, y, z])



FiniteSet((-1, 2, 0))

In [ ]:
b=charPolyR2(np.array([[-2,1],[1,-1]]))
b,b.coeffs(), type(b), 
negB(b.coeffs())

array([-3/2 + sqrt(5)/2, -3/2 - sqrt(5)/2], dtype=object)

In [ ]:
v1 = np.array([x,y,z])
v2 = np.array([z,y,x])
dotv(v1,v2), np.dot(v1,v2)

A,B = np.array([[1,2,3],[4,5,6],[7,8,9]]), np.array([[1,0,0],[1,-1,0],[-1,-1,1]])
#A,B = Amat(),Amat()
A,B
dotv(v1,v2), np.dot(v1,v2)

(2*x*z + y**2, 2*x*z + y**2)

In [ ]:
Plhs =  x1 + 2*x2 - x3 
Peqls = 0
P = Plhs-Peqls
l = [1+t,1+t,1]
subdict =  tuple(zip(P.as_terms()[1], l))
t_val = sym.solve(P.subs(subdict))

P_perp_v = [P.as_coefficients_dict()[i] for i in P.as_terms()[1]]
P_inter_l = [ i.subs(t,t_val[0]) if type(i) == sym.Add else i for i in l]
#subdict
#type(P)
#subdict
subdict,sym.solve(P.subs(subdict)),t_val, P_perp_v#P.as_coefficients_dict()#[1]#['x2']#.values()
#P.subs()
#P.subs(-sym.solve(P)[0])
#sym.Poly
check_Perp = lambda v,w : True if np.dot(v,w) == 0 else False

check_Perp(P_inter_l,[1,2,-1])

#type([1,2,1])


True

In [ ]:
p1 = sym.Plane(sym.Point3D(1, 2, 3), sym.Point3D(4, 5, 6), sym.Point3D(0, 2, 0))
p2 = p1.equation()
p1,p2

(Plane(Point3D(1, 2, 3), (-9, 6, 3)), -9*x + 6*y + 3*z - 12)

In [ ]:
A= arr 

In [45]:
#guassA = lambda:
def swapR(A,i,j):
    A[[i,j]] = A[[j,i]]
    return A

def timesR(A,i,k):
    A[i] = A[i] * k
    return A

def subR(Ri,R1):
    #print(Ri ,R1,  Ri[0]/R1[0], R1 * Ri[0]/R1[0])
    return Ri - (R1 * Ri[0]/R1[0])

indxA = lambda A : np.array([[[i[0]+1,j+1,k] for j,k in enumerate(A[i[0]])] for i in enumerate(A)])


swapMax = lambda A: swapR(A, np.argmax(A[::,0]), 0)

#cancel = lambda A: np.apply_along_axis(addR, 1, A[1:], A[0])

def cancel(A):
    for i ,j in enumerate(A):
        print(A[i][0])
        if A[i][0] != 0:
            np.apply_along_axis(subR, 1, A[1:], A[0])
            print(A)
            return A
#swapR(A,1,2)
#timesR(A,0,3)

#[1:,]

In [49]:
A#swapR(A,0,1)



⎡1  2  3⎤
⎢       ⎥
⎢3  6  2⎥
⎢       ⎥
⎣2  2  1⎦

In [ ]:
P23 = np.array([[1,0,0],[0,0,1],[0,1,0]])
P12 = np.array([[0,1,0],[0,0,1],[1,0,0]])

A,indxA(A)[1][2][-1]

A,dotA(P23.T,A), dotA(P23,A)
P12,transA(P12), P23, transA(P23)

(array([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]]),
 array([[0, 0, 1],
        [1, 0, 0],
        [0, 1, 0]]),
 array([[1, 0, 0],
        [0, 0, 1],
        [0, 1, 0]]),
 array([[1, 0, 0],
        [0, 0, 1],
        [0, 1, 0]]))

In [ ]:

#def perm(A, i,j):
#    Pij = np.zeros_like(A)
perm = lambda I, i, j: swapR(I,i,j)
#s  = np.zeros_like(A)[1][2]+1
I(A)

def E (A, i,j):
     E = I(A)
     print(A[i][0], A[j][0])
     E[i][j] =  (-A[i] / A[j])[j]
     print(f'E{i+1}{j+1} = {E}')
     return np.array(E)

def elim(A):
     #np.apply_along_axis()
     U = A
     #[[(dotA(U ,E(U,i,j)))for j in enumerate(A[i[0]])] for i in enumerate(A)]

          
     return [[ (dotA(U ,E(U,i[0],j[0]))) for i in enumerate(A)]]
#lst  =[1,2,3]
'''i if i[2] !=0 else for i in indxA(A):
     print(i)'''
#col1 = 
#for i in 
#e23 = E(A,2,1)
#A,A[1:-1, 1:-1]#, E(A,2,1),   dotA( E(A,2,1) , A), dotA(dotA(e23, A), dotA( E( dotA( E(A,2,1), A), 3, 1),A))
#elim(A)
#elim = np.array(lambda A: map(E(),A[1:]))

#elim(A)

ValueError: Matrix([[1, 2, 3], [3, 6, 2], [2, 2, 1]]) is not an integer

In [ ]:
def E2 (A ):#, piv):
     E = I(A)
     print(A.shape[0])
     '''for j in range(A.shape(0)):

          print(A[i][0], A[j][0])
          E[i][j] =  (-A[piv] / A[j])
          print(f'E{i+1}{j+1} = {E}')
      '''    
     return np.array(E)


#list(map(E2,indxA(A),))

function ToRowEchelonForm(Matrix M) is
    nr := number of rows in M
    nc := number of columns in M
    
    for 0 ≤ r < nr do
        allZeros := true
        for 0 ≤ c < nc do
            if M[r, c] != 0 then
                allZeros := false
                exit for
            end if
        end for
        if allZeros = true then
            In M, swap row r with row nr
            nr := nr - 1
        end if
    end for
    
    p := 0
    while p < nr and p < nc do
        label nextPivot:
            r := 1
            while M[p, p] = 0 do 
                if (p + r) <= nr then
                    p := p + 1
                    goto nextPivot
                end if
                In M, swap row p with row (p + r)
                r := r + 1
            end while
            for 1 ≤ r < (nr - p) do 
                if M[p + r, p] != 0 then
                    x := -M[p + r, p] / M[p, p]
                    for p ≤ c < nc do
                        M[p + r, c] := M[p , c] * x + M[p + r, c]
                    end for
                end if
            end for
            p := p + 1
    end while
end function


In [ ]:
A=np.array([[0,0,0],[0,2,3],[0,0,0],[5,6,4],[0,0,2]])
A,len(A)

(array([[0, 0, 0],
        [0, 2, 3],
        [0, 0, 0],
        [5, 6, 4],
        [0, 0, 2]]),
 5)

In [ ]:
[i for i in range(3,5)]
A,np.array(A[2:5]),sum(sum(np.array(A[2:5])))

(array([[0, 0, 0],
        [0, 2, 3],
        [0, 0, 0],
        [5, 6, 4],
        [0, 0, 2]]),
 array([[0, 0, 0],
        [5, 6, 4],
        [0, 0, 2]]),
 17)

In [ ]:

sum([sum(A[i]) for i in range(3,5)]),A[4]

(17, array([0, 0, 2]))

In [ ]:
print(A)
def swap_r0(A):
    nr = len(A)
    nswap=0
    
    for i in range(nr):
        #print(i,nr)
        print(A[i])
         #nswap = nr-1
        
        if sum(sum(np.array(A[i:nr]))) == 0:
            print(i, nr,'sum0')
            return A
        
        elif sum(A[i]) == 0:#and sum(A[nswap]) != 0:
            nswap += 1
           
            swapR(A,i,-nswap)
            
            print(i,nswap) 
pivlist =[]
def piv(A,row):
    loc = list(enumerate(A[row]))
    #print(loc,loc[1][1])
    i=0
    while i <= len(loc) -1:
        if loc[i][1] != 0:
            pivlist.append(loc[i] )
            return

        else:
            #print(i) 
            i +=1

def ref(A):
    A = swap_r0(A)
    
    for i in len(A):


piv(A,0),pivlist#,swap_r0(A)

IndentationError: expected an indented block (<ipython-input-21-c908318a6e73>, line 41)

In [ ]:
A, len(A)

(array([[0, 0, 0],
        [0, 2, 3],
        [0, 0, 0],
        [5, 6, 4],
        [0, 0, 2]]),
 5)

In [ ]:
def ref(A):
        
        #comb = I(A)
        A = np.array(A)
        for j in range(A.shape[1]-1) :
                
                i=0
                while i+1+j <= A[i:, j:].shape[0]:
                        print(A, i+1+j, j)
                        comb = dotA(E(A, i+1+j ,j), comb)
                        #print(j,i+1+j,comb,A[i:, j:])
                        i+=1
                        
                A = dotA(comb, A)
                print(A, comb)
        #i += 2       
#A, A[1:]
#map(E(A),A[1:])
#cancel(A)
#np.argmax(A[::,0])
#max(A[::,0])
#swapMax(A)
A, ref(A)

[[1 2 3]
 [3 6 2]
 [2 2 1]] 1 0


NameError: name 'E' is not defined

In [ ]:
t1= np.array(([1,0,0],[-4,1,0],[0,0,1]))
t2 = np.array(([1,0,0],[0,1,0],[-7,0,1]))
t3 = np.array(([1,0,0],[0,1,0],[0,-2,1]))
res1 = dotA(t1,A)
res2 = dotA(t2, res1)
res3 = dotA(t3, res2)
#res3 = 
A, res1, res2,res3

ValueError: operands could not be broadcast together with shapes (3,) (5,) 

In [ ]:
#E,A= Amat(),Amat()
vA = np.array(([a,b,c],[d,e,f],[g,h,i]))#, np.array([[e,f],[g,h]])
vB = np.array(([a0,a1,a2],[a3,a4,a5],[a6,a7,a8]))
I(vA),vA

#i,A,B, dotA(vA, vB), dotA(A,B)

#detA3(A),
#np.linalg.det(A), 
#E, A, dotA(E,A)
#vA, vB, detA2(vA), detA2(vB), detA2(dotA(vA,vB))
dotA(vA,vB)

array([[a*a0 + a3*b + a6*c, a*a1 + a4*b + a7*c, a*a2 + a5*b + a8*c],
       [a0*d + a3*e + a6*f, a1*d + a4*e + a7*f, a2*d + a5*e + a8*f],
       [a0*g + a3*h + a6*i, a1*g + a4*h + a7*i, a2*g + a5*h + a8*i]],
      dtype=object)

In [ ]:
A = Amat()
v = Amat()
#A,v,dotA(A, v)
vA, vB, detA(dotA(vA,vB)), detA(vA), detA(vB)

NameError: name 'detA' is not defined

In [ ]:

A,v

In [ ]:
dotA(A,v)


In [ ]:

np.linalg.tensorsolve(A,[1,1,1])
np.linalg.cond(A), np.linalg.det(A),detA3(A)

In [ ]:
A[0],B[::,0], len(B.T)
[print(i) for i in B[::,1]]
#[i for i in range(B.shape[1])]

In [ ]:
dotA(Amat(),Amat())

In [ ]:
ide(A)

140201258445392

In [ ]:
A[1]

In [ ]:
 v ,w =n(),  n()
 
 dotv(v,w)

In [ ]:


def plotv (u,v= np.zeros_like(u), w =np.zeros_like(u) ):
    M = np.array([u,v,w])
    print(M, M.T.shape)
    rows,cols = M.T.shape
    #maxes = 1.2*np.amax(abs(M), axis = 0)

    for i,l in enumerate(range(0,cols)):
        xs = [0,M[i,0]]
        ys = [0,M[i,1]]
        print(xs,ys, )
        plt.plot(xs,ys)

    plt.plot(0,0,'ok') #<-- plot a black point at the origin
    plt.axis('equal')  #<-- set the axes to the same scale
    #plt.xlim([-maxes[0],maxes[0]]) #<-- set the x axis limits
    #plt.ylim([-maxes[1],maxes[1]]) #<-- set the y axis limits
    plt.axhline(color='black')
    plt.axvline(color='black')
    plt.legend(['V'+str(i+1) for i in range(cols)]) #<-- give a legend
    plt.grid(b=True, which='major') #<-- plot grid lines
    
    return plt.show()

def plotv1(u,v= np.zeros_like(u), w =np.zeros_like(u) ):
    M = np.array([u,v,w])
    rows,cols = M.T.shape
    y = map(lambda M, i:   (
                                [0, M[[M[0:i], 0]]], 
                                [0, M[[M[0:i], 1]]] ),
                                (M, range(0,cols)))

    print(range(0,3,1), M[0,0],y)

    #plt.plot(y)

    plt.plot(0,0,'ok') #<-- plot a black point at the origin
    plt.axis('equal')  #<-- set the axes to the same scale
    #plt.xlim([-maxes[0],maxes[0]]) #<-- set the x axis limits
    #plt.ylim([-maxes[1],maxes[1]]) #<-- set the y axis limits
    plt.axhline(color='black')
    plt.axvline(color='black')
    #plt.legend(['V'+str(i+1) for i in range(cols)]) #<-- give a legend
    plt.grid(b=True, which='major') #<-- plot grid lines
    return y

#np.zeros
u = np.array([-.6,.8])
v = np.array([4,3])
w = np.array([1,2])
plotv(w,v)#,dotv(u,v))

In [ ]:
dotv(w,v),unitv(v)


#lenv(v)

In [ ]:
#Q 1.2a
v = np.array([3,4])
w = np.array([2,3])
schwarz(v,w), triIneq(v,w), cosθ(v,w), degθ(cosθ(v,w))

In [ ]:
#Q1.2b
unitv(v)

In [ ]:
#prob set 1.2
#q1 
u = np.array([-.6,.8])
v = np.array([4,3])
w = np.array([1,2])

dotv(u,v), dotv(u,w), dotv(u,v+w), dotv(v,w)

In [ ]:
#q2
lenv(u),lenv(v),lenv(w), schwarz(u,v), schwarz(v,w)

In [ ]:
dotv(unitv((v-w)),unitv((v+w))),unitv((v-w)), unitv(v+w),v-w, v+w


In [ ]:
v, w = np.array([-1, -2,3]), np.array([4,0,-8])
dotv(v,w) 

In [ ]:
-v,v

In [ ]:
l = ( x/sym.sqrt(y) )**2
m = (x/sym.sqrt(y) )
sym.sqrt(l), m

In [ ]:
if sym.sqrt(l) ==  m:
    print (f'True {sym.sqrt(l)} = {m}:')
else:
    print(False)

In [ ]:
l = lambda x, y: ( x/sym.sqrt(y) )**2
m = lambda x, y: (x/sym.sqrt(y) )

In [ ]:
sym.sqrt(l(3,4)), m(3,4)


In [ ]:
v, w = np.array([2,2,-1]), np.array([2,-1,2])
dotv(v,w), lenv(v),lenv(w), cosθ(v,w),degθ(cosθ(v,w))


In [ ]:
n= lambda : [int(i) for i in input()] 
#print(type(a()),a())

#x = 


In [ ]:


dimA

In [ ]:
dimA = np.zeros((n()))
A = np.reshape(np.asarray(n()), dimA.shape)
A

In [ ]:
#dotA = 

In [ ]:
np.linalg.inv(A)

In [ ]:
#crossA = lambda A:
len(A) 